# Graded Lab Assignment 2: Evaluate classifiers (10 points)
 
In this assignment you will optimize and compare the perfomance of a parametric (logistic regression) and non-parametric (k-nearest neighbours) classifier on the MNIST dataset.

Publish your notebook (ipynb file) to your Machine Learning repository on Github ON TIME. We will check the last commit on the day of the deadline.  

### Deadline Friday, November 17, 23:59.

This notebook consists of three parts: design, implementation, results & analysis. 
We provide you with the design of the experiment and you have to implement it and analyse the results.

### Criteria used for grading
* Explain and analyse all results.
* Make your notebook easy to read. When you are finished take your time to review it!
* You do not want to repeat the same chunks of code multiply times. If your need to do so, write a function. 
* The implementation part of this assignment needs careful design before you start coding. You could start by writing pseudocode.
* In this exercise the insights are important. Do not hide them somewhere in the comments in the implementation, but put them in the Analysis part
* Take care that all the figures and tables are well labeled and numbered so that you can easily refer to them.
* A plot should have a title and axes labels.
* You may find that not everything is 100% specified in this assignment. That is correct! Like in real life you probably have to make some choices. Motivate your choices.


### Grading points distribution

* Implementation 5 points
* Results and analysis 5 points

## Design of the experiment

You do not have to keep the order of this design and are allowed to alter it if you are confident.
* Import all necessary modules. Try to use as much of the available functions as possible. 
* Use the provided train and test set of MNIST dataset.
* Pre-process data eg. normalize/standardize, reformat, etc.           
  Do whatever you think is necessary and motivate your choices.
* (1) Train logistic regression and k-nn using default settings.
* Use 10-fold cross validation for each classifier to optimize the performance for one parameter: 
    * consult the documentation on how cross validation works in sklearn (important functions:             cross_val_score(), GridSearchCV()).
    * Optimize k for k-nn,
    * for logistic regression focus on the regularization parameter,
* (2) Train logistic regression and k-nn using optimized parameters.
* Show performance on the cross-validation set for (1) and (2) for both classifiers: 
    * report the average cross validation error rates (alternatively, the average accuracies - it's up to you) and standard deviation,
    * plot the average cross valildation errors (or accuracies) for different values of the parameter that you tuned. 
* Compare performance on the test set for two classifiers:
    * produce the classification report for both classifiers, consisting of precision, recall, f1-score. Explain and analyse the results.
    * print confusion matrix for both classifiers and compare whether they missclassify the same  classes. Explain and analyse the results.
* Discuss your results.
* BONUS: only continue with this part if you are confident that your implemention is complete 
    * tune more parameters of logistic regression
    * add additional classifiers (NN, Naive Bayes, decision tree), 
    * analyse additional dataset (ex. Iris dataset)

In [ ]:
from sklearn.datasets import load_digits
import numpy
import os
import struct
import math
import operator 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.sparse

# load MNIST dataset and split in train and test set.
# the training set consists of 1500 images of 8x8 pixels, meaning that there are 64 features in total
# the test set consists of 297 images
# the total amount of images are 1797, 8x8 pixels
# input layer is a 1x64 vector, the parameter matrix is 10x64 

# The images are stored in byte format, and we will read them into numpy arrays 
# that we will use to train and test our implementation
digits = load_digits()

# training and test set images
Xtrain = numpy.reshape(digits.images[:1500],(1500,64))
Xtest = numpy.reshape(digits.images[1500:],(297,64))

# training and test set labels; the hand-written numbers range from 0 to 9. Thus there are 10 labels
Ytrain = digits.target[:1500]
Ytest = digits.target[1500:]

# K-NEAREST NEIGHBORS ALGORITHM
def distance(instance1, instance2):
    distance = 0
    # both instances are transformed into an array instead of a matrix 
    # to make calculation easier
    instance1 = numpy.ndarray.flatten(instance1)
    instance2 = numpy.ndarray.flatten(instance2)
    length = len(instance1) # the length is 64 for all instances
    for x in range(length):
        distance += pow((instance1[x] - instance2[x]), 2)
    return math.sqrt(distance)

# The following KNN algorithm can predict the label of an instance. I have 
# decide that you can choose on which set you want the algorithm to be trained,
# which instance you want to predict the label from and also how many neighbors
# you want to take into consideration. These are all variables, so feel free
# to make this decision yourself. 
def KNN(training_set, training_target_set, instance_x, target_x, k):
    distances = []
    targets = []
    for i in range(len(training_set)):
        dist = distance(instance_x, training_set[i])
        # distances is the list with all the distances between instance_x 
        # and all the instances in the training_set
        distances.append(dist)
    for j in range(len(training_target_set)):
        # enumerates the labels together with the training_set
        targets.append(training_target_set[j])
    # combined puts the list of distances and targets in one list
    combined = numpy.vstack((distances, targets)).T
    # sort does a sorting only looking at the distances in combined
    # sorting from smallest distances to largest 
    sort = sorted(combined, key=operator.itemgetter(0))
    votes = []
    # k decides how many nearest neighbors you want to look at
    for i in range(k):
        vote = sort[k][1]
        votes.append(vote)
    counts = numpy.bincount(votes)
    prediction = numpy.argmax(counts)
    # verifies whether your prediction is true or false
    if prediction == target_x:
        return True, target_x
    if prediction != target_x:
        return False

# I have decided that the output of this algorithm the label of the instance is
# only if the algorithm has made an accurate prediction. If the prediciton is
# false, the algorithm will return False. This means that you have to adjust your k

# An example of how to run the algorithm is:
KNN(Xtrain, Ytrain, digits.images[1532], digits.target[1532], 10)
KNN(Xtrain, Ytrain, digits.images[1600], digits.target[1600], 30)

#LOGISTIC REGRESSION
mnist = digits
batch = Xtrain
tb = Xtest


def getLoss(w,x,y,lam):
    m = x.shape[0] # the number of training examples
    y_mat = oneHotIt(y) #Next we convert the integer class coding into a one-hot representation
    scores = np.dot(x,w) #computation of the raw class scores given our input and current weights
    prob = softmax(scores) #Next we perform a softmax on these scores to get their probabilities
    loss = (-1 / m) * np.sum(y_mat * np.log(prob)) + (lam/2)*np.sum(w*w) #We then find the loss of the probabilities
    grad = (-1 / m) * np.dot(x.T,(y_mat - prob)) + lam*w #And compute the gradient for that loss
    return loss,grad

def oneHotIt(Y):
    m = Y.shape[0]
    #Y = Y[:,0]
    OHX = scipy.sparse.csr_matrix((np.ones(m), (Y, np.array(range(m)))))
    OHX = np.array(OHX.todense()).T
    return OHX

def softmax(z):
    z -= np.max(z)
    sm = (np.exp(z).T / np.sum(np.exp(z),axis=1)).T
    return sm

def getProbsAndPreds(someX):
    probs = softmax(np.dot(someX,w))
    preds = np.argmax(probs,axis=1)
    return probs,preds

x = Xtrain
y = Ytrain
w = np.zeros([x.shape[1],len(np.unique(y))])
lam = 1
iterations = 1000
learningRate = 1e-5
losses = []
for i in range(0,iterations):
    loss,grad = getLoss(w,x,y,lam)
    losses.append(loss)
    w = w - (learningRate * grad)
print(loss)

plt.plot(losses)

def getAccuracy(someX,someY):
    prob,prede = getProbsAndPreds(someX)
    accuracy = sum(prede == someY)/(float(len(someY)))
    return accuracy

# here we can print the training accuracy and the test accuracy
print('Training Accuracy:', getAccuracy(x,y))
print('Test Accuracy:', getAccuracy(Xtest,Ytest))

# we can also play around and try to visulize some number such as 3 and 1 
classWeightsToVisualize = 3
plt.imshow(scipy.reshape(w[:,classWeightsToVisualize],[8,8]))

classWeightsToVisualize = 1
plt.imshow(scipy.reshape(w[:,classWeightsToVisualize],[8,8]))

## Results and analysis of the experiment

In [ ]:
from __future__ import print_function
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn import datasets
from skimage import exposure
import numpy as np
import imutils
import cv2
from sklearn import datasets, svm, metrics
import matplotlib.pyplot as plt


# KNN ALGORITHM OPTIMIZATION OF K AND CROSS VALIDATION
mnist = digits
 
# take the MNIST data and construct the training and testing split, using 83.5% of the
# data for training and 16.5% for testing
(trainData, testData, trainLabels, testLabels) = train_test_split(np.array(mnist.data), mnist.target, test_size=0.165, random_state=42)
 
# to use a 10-fold cross-validation we that 10% from the training set 
(trainData, valData, trainLabels, valLabels) = train_test_split(trainData, trainLabels, test_size=0.1, random_state=84)
 
# here the length of the sets are shown 
print("training data points: {}".format(len(trainLabels)))
print("validation data points: {}".format(len(valLabels)))
print("testing data points: {}".format(len(testLabels)))

# initialize the values of k for our k-Nearest Neighbor classifier along with the
# list of accuracies for each value of k
# I only want to look at a number of k that is odd, because if k is odd, you know
# for sure that one label is more frequent than any other label. 
kVals = range(1, 30, 2)
accuracies = []
 
# loop over various values of `k` for the k-Nearest Neighbor classifier
for k in xrange(1, 30, 2):
    # train the kNN classifier with the current value of `k`
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(trainData, trainLabels)
 
    # evaluate the model and update the accuracies list
    score = model.score(valData, valLabels)
    print("k=%d, accuracy=%.2f%%" % (k, score * 100))
    accuracies.append(score)
 
    # find the value of k that has the largest accuracy
i = np.argmax(accuracies)
print("k=%d achieved highest accuracy of %.2f%% on validation data" % (kVals[i], accuracies[i] * 100))

# re-train our classifier using the best k value and predict the labels of the
# test data
model = KNeighborsClassifier(n_neighbors=kVals[i])
model.fit(trainData, trainLabels)
predictions = model.predict(testData)
 
# show a final classification report demonstrating the accuracy of the classifier
# for each of the digits
print("EVALUATION ON TESTING DATA")
print(classification_report(testLabels, predictions))

for i in np.random.randint(0, high=len(testLabels), size=(5,)):
    image = testData[i]
    prediction = model.predict(image)[0]
 
    # In order to see the image 'better', I reshape the image
    image = image.reshape((8, 8)).astype("uint8")
    image = exposure.rescale_intensity(image, out_range=(0, 255))
    image = imutils.resize(image, width=32, inter=cv2.INTER_CUBIC)
 
    # here the prediction is showed 
    print("I think that digit is: {}".format(prediction))
    cv2.imshow("Image", image)
    cv2.waitKey(0)
    
#LOGISTIC REGRESSION
print(__doc__)

# The MNIST dataset
digits = datasets.load_digits()
images_and_labels = list(zip(digits.images, digits.target))
for index, (image, label) in enumerate(images_and_labels[:4]):
    plt.subplot(2, 4, index + 1)
    plt.axis('off')
    plt.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title('Training: %i' % label)

# Once again I reshape de images to make computation easier 
n_samples = len(digits.images)
data = digits.images.reshape((n_samples, -1))

# I decided to use a support vector classifier to make it easy to print 
# a confusion matrix as well as precision, recall, f1-score and support table
classifier = svm.SVC(gamma=0.001)

# We learn the digits on the first half of the digits
classifier.fit(data[:n_samples // 2], digits.target[:n_samples // 2])

# Now predict the value of the digit on the second half:
expected = digits.target[n_samples // 2:]
predicted = classifier.predict(data[n_samples // 2:])

print("Classification report for classifier %s:\n%s\n"
      % (classifier, metrics.classification_report(expected, predicted)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

images_and_predictions = list(zip(digits.images[n_samples // 2:], predicted))
for index, (image, prediction) in enumerate(images_and_predictions[:4]):
    plt.subplot(2, 4, index + 5)
    plt.axis('off')
    plt.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title('Prediction: %i' % prediction)

plt.show()